In [ ]:
import torch
import torchvision
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

: 

In [ ]:
def create_data_generator(batch_size = 32, root = "data"):
    # download trainable MNIST dataset
    train_dataset = torchvision.datasets.MNIST(root = root,
                                             train = True,
                                             transform = torchvision.transforms.ToTensor(),
                                             download = True)
    # download testable MNIST dataset
    test_dataset = torchvision.datasets.MNIST(root = root,
                                             train = False,
                                             transform = torchvision.transforms.ToTensor(),
                                             download = True)
    # dataset = train_dataset
    # shuffle true to ensure randomness
    # num_workers = 2 to ensure faster loading
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size = batch_size,
                                             shuffle = True)
    # not shuffled to ensure consistency during evaluation
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size = batch_size,
                                             shuffle = False)
    return train_loader, test_loader

In [ ]:
class CharacterNetwork(nn.Module): # inherit from neural network module
    def __init__(self, hidden_dims = [128]):
        super(CharacterNetwork, self).__init__()
        self.network_dims = [28 * 28] + hidden_dims + [10]
        self.layers = []
        for i, dim in enumerate(self.network_dims[1:]): # iterate through network dimensions to create layers
            prev_dim = self.network_dims[i]
            dense = nn.Linear(in_features = prev_dim, out_features = dim, bias = True) # linear layer
            if i < len(self.network_dims[1:]) - 1:
                activation = nn.Sigmoid() # Hidden Layer
            else:
                activation = nn.Softmax(dim=1) # Last Layer
            self.layers += [dense, activation]
            
        self.layers = nn.ModuleList(self.layers)

    def forward(self, x): # forward pass
        # input x is [B, 1, 28, 28] where B is the batch size, 1 is num channels (grayscale), 28x28 is image size
        size = x.size()
        x = x.reshape(size[0], -1) # Flatten images to [B, 28 * 28]
        for layer in self.layers[:-1]:
            x = layer(x)
        if self.training == False:
            x = self.layers[-1](x)
        return x

In [ ]:
cuda = torch.cuda.is_available()
batch_size = 32 # change batch size probably
train_loader, test_loader = create_data_generator(batch_size)
model = CharacterNetwork() # create an instance of the model
print(model)

In [ ]:
def train(model, loader, n_epochs = 1, learning_rate = 0.1): # train the model pass in model, dataloader, number of epochs, and learning rate
    cuda = torch.cuda.is_available() # check if cuda is available
    if cuda:
        model.cuda() # move model to GPU

    optim = torch.optim.SGD(params = model.parameters(), lr = learning_rate, momentum=0.9) # initialize stochastic gradient descent optimizer
    loss_fn = nn.CrossEntropyLoss() # initialize cross entropy loss function (imported from torch.nn)
    log_losses = []
    log_accuracy = []
    model.train()
    
    for epoch in range(1, n_epochs + 1): # training loop for n_epochs
        for idx, (images, labels) in enumerate(loader): # iterate through the dataloader to get batches of images and labels
            if cuda:
                images = images.cuda() # move images to GPU
                labels = labels.cuda() # move labels to GPU
                
            outputs = model(images) # pass images through the model to get outputs
            
            loss = loss_fn(outputs, labels) # calculate loss)
            
            optim.zero_grad() # zero out / reset gradients
            loss.backward() # backpropagate -> compute gradients
            optim.step() # update weights
            
            
            
            predictions = torch.argmax(outputs, 1)
            accuracy = (predictions == labels).float().mean() * 100.
            
            loss = loss.item() # Convert to Python Scalar
            accuracy = accuracy.item() # Convert to Python Scalar
            log_losses.append(loss)
            log_accuracy.append(accuracy)
            
            if idx % 500 == 0: # print loss and accuracy every 500 iterations
                print("Epoch [%d/%d]. Iter [%d/%d]. Loss: %0.2f. Accuracy: %0.2f" % (epoch, n_epochs, idx + 1, len(loader), loss, accuracy))

    return model, log_losses, log_accuracy

In [ ]:
# Training Model
model, losses, accuracies = train(model, train_loader, n_epochs = 1, learning_rate = 0.1) # change epochs, learning rate here

# Saving Model
print("Saving Model to CharacterModel.pth")
torch.save(model.state_dict(), "CharacterModel.pth")

# Visualize Training Losses
fig, (ax1, ax2) = plt.subplots(1, 2)
_ = ax1.plot(losses)
_ = ax1.set_title("Training Losses")
_ = plt.figure()
_ = ax2.plot(accuracies)
_ = ax2.set_title("Training Accuracy")

In [ ]:
def evaluate(model, loader): # evaluate takes in the trained model and the test_loader
    cuda = torch.cuda.is_available()
    if cuda:
        model.cuda()
        
    model.eval() # set model to evaluation mode
    model.load_state_dict(torch.load("CharacterModel.pth")) # load the trained model

    total_accuracy = 0.0
    for idx, (images, labels) in enumerate(test_loader): # iterate through the test_loader (test set of images and labels)
        if cuda:
            images = images.cuda() # move test images to GPU
            labels = labels.cuda() # move test labels to GPU
        
        outputs = model(images) # pass test images through the model to get outputs
        
        predictions = torch.argmax(outputs, 1) #
        accuracy = (predictions == labels).float().mean() * 100.
        
        accuracy = accuracy.item() # Convert to Python Scalar
        
        total_accuracy += accuracy
        
        if idx % 200 == 0:
            print("Iter [%d/%d]. Accuracy: %0.2f" % (idx + 1, len(test_loader), accuracy))
        
    print("Final Accuracy: %0.2f" % (total_accuracy / len(test_loader)))

In [ ]:
# Evaluate Model 
evaluate(model, test_loader)

In [ ]:
def nms(boxes, scores, threshold): # non-maximum suppression
    # non maximum supression to remove overlapping boxes (takes the most confident box)
        # boxes are of format: [x1, y1, x2, y2]
    if len(boxes) == 0:
        return []

    # Convert boxes to NumPy array
    boxes = np.array(boxes)

    # Grab the coordinates of the bounding boxes
    x1 = boxes[:, 0] # [:, 0] does what? 
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # Compute the area of the bounding boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort the bounding boxes by the confidence score
    indices = np.argsort(scores)[::-1]

    pick = []

    while len(indices) > 0:
        # Grab the index of the bounding box with the highest score
        i = indices[0]
        pick.append(i)

        # Compute the intersection over union (IoU)
        xx1 = np.maximum(x1[i], x1[indices[1:]])
        yy1 = np.maximum(y1[i], y1[indices[1:]])
        xx2 = np.minimum(x2[i], x2[indices[1:]])
        yy2 = np.minimum(y2[i], y2[indices[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        iou = (w * h) / (areas[i] + areas[indices[1:]] - w * h)

        # Remove the bounding boxes with IoU greater than the threshold
        indices = np.delete(indices, np.concatenate(([0], np.where(iou > threshold)[0] + 1)))

    return pick
    
def find_contours(dimensions, img) : # find contours in the image, filters based on size, and applies non-maximum suppression
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    
    # Check top 15 largest contours for character
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    

    target_contours = []
    img_res = []
    boxes = []
    scores = []
    for cntr in cntrs : # iterate through the contours
        # detects contour in binary image and returns the coordinates of rectangle enclosing it
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
        
        # checking the dimensions of the contour to filter out the characters by contour's size
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            
            char_copy = np.zeros((60,40))
            # extracting each character from the image using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            # Make the aspect ratio of the detected character to be 1:2
            char = cv2.resize(char, (20, 40))
            # Padding the character to fit the size of 28x28
            char_copy[10:50, 10:30] = char
            
            # Reshape the character to be fed into the model
            boxes.append([intX, intY, intX+intWidth, intY+intHeight])
            scores.append(intWidth*intHeight)
            img_res.append(char_copy) # List that stores the character's binary image (unsorted)

    boxes = np.array(boxes)
    scores = np.array(scores)
    keep = nms(boxes, scores, 0.1)


    # return filtered character images and their bounding boxes coordinates
    return [img_res[i] for i in keep], [boxes[i] for i in keep]

In [ ]:

def detect_characters(image_path, model): # detect and label characters in an image taking in the image path and the trained model
    cuda = torch.cuda.is_available()
    if cuda:
        model.cuda() # move model to GPU
    model.eval()
    model.load_state_dict(torch.load("CharacterModel.pth")) # load the trained model
    
    image = cv2.imread(image_path)
    image = img_lp = cv2.resize(image, (333, 75))

    
    img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY) # convert to grayscale
    _, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) # convert to binary
    img_binary_lp = cv2.erode(img_binary_lp, (3,3)) # erosion and dialation to remove noise
    img_binary_lp = cv2.dilate(img_binary_lp, (3,3))

    # Get dimensions of license plate (binary image)
    LP_HEIGHT = img_binary_lp.shape[0]
    LP_WIDTH = img_binary_lp.shape[1]

    # Make borders white
    img_binary_lp[0:3,:] = 255
    img_binary_lp[:,0:3] = 255
    img_binary_lp[72:75,:] = 255
    img_binary_lp[:,330:333] = 255

    # Estimations of character contours sizes of cropped license plates
        # contours are just the green boxes around the characters
    dimensions = [0, LP_WIDTH/2, LP_HEIGHT/6, 3*LP_HEIGHT/3]

    
    # Get contours within cropped license plate
    char_list, boxes = find_contours(dimensions, img_binary_lp)
    for idx, char in enumerate(char_list): # iterates over detected character contours
        char = cv2.resize(char, (28, 28)) # resize the character to 28x28
        char = torch.from_numpy(char).resize(1, 1, 28, 28).float() # converts to pytorch tensor to be fed into the model
        if cuda:
            char = char.cuda() # move character (represented as a tensor) to GPU
        
        outputs = model(char) # pass character through the model to get outputs
        
        pred = torch.argmax(outputs, 1)[0].item() # get the predicted character class (0-9, A-Z)
        # print(pred)

        x1, y1, x2, y2 = boxes[idx] # get the bounding box coordinates of the character
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1) # draw a green rectangle around the character (detected contour)
        image = cv2.putText(image, f'{pred}', (x1+2, y1+12), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA) # put the predicted character class (ex: 'H') on the image

    # display annotated image with green boxes and their predicted classes using matplotlib
    plt.figure() 
    plt.imshow(image[..., ::-1])
    plt.axis("off")
        

In [ ]:
# detect_characters("data/example1.jpeg", model)
# detect_characters("data/example2.png", model)
# detect_characters("data/example3.png", model)